In [4]:
import matplotlib.pyplot as plt 
import pandas as pd
from bs4 import BeautifulSoup
import requests

Matplotlib is building the font cache; this may take a moment.


In [5]:
browse = "https://miamiuniversityartmuseum.omeka.net/items/browse"

In [6]:
response = requests.get(browse)
if response.status_code != 200:
    print(f"Failed to retrieve data from the URL. Status code: {response.status_code}")
    exit()

soup = BeautifulSoup(response.text, 'html.parser')

In [7]:
link_on_pages = []
for link in soup.find_all('a'):

    href = link.get('href')
    if href and href.startswith('/items/show/'):
        #print(href)
        link_on_pages.append(href)

print(link_on_pages)

['/items/show/20713', '/items/show/20712', '/items/show/20711', '/items/show/20710', '/items/show/20709', '/items/show/20708', '/items/show/20707', '/items/show/20706', '/items/show/20705', '/items/show/20704']


In [8]:
for link in link_on_pages:
    full_link = "https://miamiuniversityartmuseum.omeka.net" + link
    print(full_link)
    response = requests.get(full_link)
    if response.status_code != 200:
        print(f"Failed to retrieve data from the URL. Status code: {response.status_code}")
        exit()


soup = BeautifulSoup(response.text, 'html.parser')

https://miamiuniversityartmuseum.omeka.net/items/show/20713
https://miamiuniversityartmuseum.omeka.net/items/show/20712
https://miamiuniversityartmuseum.omeka.net/items/show/20711
https://miamiuniversityartmuseum.omeka.net/items/show/20710
https://miamiuniversityartmuseum.omeka.net/items/show/20709
https://miamiuniversityartmuseum.omeka.net/items/show/20708
https://miamiuniversityartmuseum.omeka.net/items/show/20707
https://miamiuniversityartmuseum.omeka.net/items/show/20706
https://miamiuniversityartmuseum.omeka.net/items/show/20705
https://miamiuniversityartmuseum.omeka.net/items/show/20704


In [9]:
data = {}

# Extract fields using a loop
fields = {
    'dublin-core-title': 'Title',
    'dublin-core-identifier': 'Identifier',
    'dublin-core-subject': 'Subject',
    'dublin-core-description': 'Description',
    'dublin-core-creator': 'Creator',
    'dublin-core-format': 'Format',
    'dublin-core-date': 'Date',
    'dublin-core-medium': 'Medium',
    'physical-object-item-type-metadata-donor': 'Donor',
    'item-citation': 'Citation'
}

for field_id, field_name in fields.items():
    element = soup.find('div', {'id': field_id})
    if element:
        data[field_name] = element.find('div', {'class': 'element-text'}).get_text(strip=True)

# Extract tags
data['Tags'] = [tag.get_text(strip=True) for tag in soup.find_all('a', {'rel': 'tag'})]

# Extract image URL
image_element = soup.find('div', {'id': 'item-images'})
if image_element:
    data['Image URL'] = image_element.find('a')['href']

# Extract collection link
collection_element = soup.find('div', {'id': 'collection'})
if collection_element:
    data['Collection Link'] = collection_element.find('a')['href']

# Print out the extracted data
for key, value in data.items():
    print(f"{key}: {value}\n")


Title: Palo Alto

Identifier: 2024.16

Subject: Graphic Arts-Prints

Description: Print by the title "Palo Alto" done by lithographic process on Arches 88 paper in 1978 by American artist Robert Motherwell (1915-1991) as indicated by his signature in pencil. Also marked in pencil with the edition number "20" of an edition of 40 produced. The print was published by Gemini G. E. L.Black symbol on red background.

Creator: Robert Motherwell (American, 1915-1991)

Format: PrintImage Size: 36 inches x 24 inches

Date: 1978

Medium: Lithographic process on Arches 88 paper

Donor: Gift of Jeffrey L. Horrell '75 and Rodney F. Rose

Citation: Robert Motherwell (American, 1915-1991), “Palo Alto,”Richard and Carole Cocks Art Museum at Miami University, accessed April 10, 2025,https://miamiuniversityartmuseum.omeka.net/items/show/20704.

Tags: ['20th Century', 'African Oceanic and New World Cultures', 'African Oceanic and New World Cultures-North America', 'American', 'Arches 88 paper', 'Black', '

In [23]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

MODEL = "llama3"

In [16]:
import pandas as pd
from langchain import OpenAI

In [17]:
from langchain_community.llms import Ollama
model = Ollama(model=MODEL)
model.invoke("What is the capital of france?")

'The capital of France is Paris.'

In [ ]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

chain = model | parser
chain.invoke("What is the capital of India?")  

'The capital of India is New Delhi.'

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/omeka_pdf.pdf")
# Load and split the document
pages = loader.load_and_split()
pages

[Document(metadata={'producer': 'Skia/PDF m135', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36', 'creationdate': '2025-04-10T04:19:39+00:00', 'title': 'Pink Cone · Richard and Carole Cocks Art Museum at Miami University', 'moddate': '2025-04-10T04:19:39+00:00', 'source': 'data/omeka_pdf.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='Skip to main content\nRichard and Carole Cocks Art Museum at Miami University\nMenu\nBrowse Exhibits\nBrowse Collections\nBrowse Items\nArt Museum Website\n \nSearch using this query type:\nKeyword\nBoolean\nExact match\nSearch only these record types:\n Item\n File\n Collection\n Simple Page\n Exhibit\n Exhibit Page\nAdvanced Search (Items only)\nPink Cone\nTitle\nPink Cone\nIdentiﬁer\n2024.23\nSubject\nGraphic Arts-Prints\nDescription\nPrint by the title "Pink Cone" done by process of hard ground etching printed in pink on gampi paper\nchine colle in 

In [36]:
from langchain.prompts import PromptTemplate
template = """
You are a helpful assistant at the Art Museum. 
You will be provided with a context and a question. 
Your task is to answer the question based on the context.
Context: {context}
Question: {question}
 """         

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="What is the capital of France?")

'\nYou are a helpful assistant at the Art Museum. \nYou will be provided with a context and a question. \nYour task is to answer the question based on the context.\nContext: Here is some context\nQuestion: What is the capital of France?\n '